# Домашнее задание EDA-4. СТАТИСТИЧЕСКИЕ ТЕСТЫ (HW-03)

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA-тест, проверьте данные на нормальность и на наличие корреляции. 

Для этого:

- Объедините два массива в DataFrame. 
- Обоснуйте выбор теста на корреляцию. 
- Сделайте вывод по гипотезе.
- Ответьте на вопрос: Какова разница в среднем размере мидии в зависимости от города-производителя
- Обоснуйте выбор теста. 
- Сделайте вывод по гипотезе.

In [82]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import pearsonr
from scipy.stats import ttest_ind

In [76]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

# преобразование списков в Series, объединение в DataFrame
mussels_df = pd.concat([pd.Series(petersburg, name='petersburg'), pd.Series(magadan, name='magadan')], axis=1)

# заполнение пустых значений
mussels_df['petersburg'].fillna(mussels_df['petersburg'].median(), inplace=True)
mussels_df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


Проверка данных на нормальность.

Для небольших выборок, как в нашем случае, подходит тест Шапиро-Уилка:

In [77]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

alpha = 0.05

_, p = shapiro(mussels_df)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.101
Данные распределены нормально


Для сравнения приведем тест Д’Агостино. Убеждаемся, что тест Д’Агостино работает только с выборками, где количество элементов >= 20.

In [78]:
_, p = normaltest(mussels_df)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.035
Данные распределены нормально


c:\Users\kkaza\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\stats\_stats_py.py:1772: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Для количественных данных с нормальным распределением подходит параметрический тест на корреляцию Пирсона:

In [81]:
H0 = 'В данных отсутствует зависимость между переменными'
Ha = 'В данных присутствует зависимость между переменными'

_, p = pearsonr(mussels_df['petersburg'], mussels_df['magadan'])

if p>alpha:
    print(f"{round(p, 2)} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.")
else:
    print(f"{round(p, 2)} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутствии зависимости между переменными.")
    

0.54 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутствии зависимости между переменными.


Зависимая переменная (размер раковины мидий) является количественной. Сравниваются 2 группы из разных совокупностей. Размер выборки менее 30. Следовательно, мы используем независимый T-тест:

In [80]:
H0 = 'Нет значимой разницы между размерами раковин мидий из Петербурга и Магадана.'
Ha = 'Есть значимая разница между размерами раковин мидий из Петербурга и Магадана.'

def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(mussels_df['petersburg'], mussels_df['magadan'], equal_var=True)

    p = round(test_results[1],3)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(mussels_df)


*** Результаты независимого T-теста ***
0.003 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами раковин мидий из Петербурга и Магадана.
